In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test

In [7]:
import numpy as np
import pandas as pd
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.metrics import confusion_matrix,precision_score,recall_score,classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from optuna.samplers import RandomSampler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn import metrics
import matplotlib.pyplot as plt
import optuna
import lightgbm as lgb
optuna.logging.set_verbosity(optuna.logging.WARNING)
from colorama import Fore
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
m_ = Fore.MAGENTA
r_ = Fore.RED
y_ = Fore.YELLOW

## Load Train and Test Data

In [8]:
clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
clinical

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
...,...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,13.0,0.0,Off
2611,65043_54,65043,54,4.0,8.0,11.0,1.0,Off
2612,65043_60,65043,60,6.0,6.0,16.0,1.0,Off
2613,65043_72,65043,72,3.0,9.0,14.0,1.0,Off


In [9]:
fig = px.histogram(clinical, x="visit_month", title="Count of Visit Month")

# Get the counts of visit_month values for annotations
counts = clinical['visit_month'].value_counts().sort_index()

# Add line width and color to the histogram bars
fig.update_traces(marker=dict(line=dict(width=1, color='black')))

fig.show()


It is obvious that vist month from 0-4 has the most clinical data

In [10]:
# Filter the data to include only visit_month values between 0 and 4
filtered_data = clinical[(clinical['visit_month'] >= 0) & (clinical['visit_month'] <= 4)]

fig = px.histogram(filtered_data, x="visit_month", title="Count of Visit Month (0 to 4)")

# Get the counts of visit_month values for annotations
counts = filtered_data['visit_month'].value_counts().sort_index()

# Add line width and color to the histogram bars
fig.update_traces(marker=dict(line=dict(width=1, color='black')))

# Add counts as text above the bins
fig.update_traces(text=counts.values, textposition='outside')

fig.show()


Our baseline month will be 0, so now there are more data available for the third month compared to other time points, making it more suitable for a robust analysis.

## Training only first month 

In this competition, the goal is to predict the course of Parkinson's disease using protein abundance data. The dataset consists of protein abundance values derived from mass spectrometry readings of cerebrospinal fluid (CSF) samples gathered from several hundred patients. Each patient contributed several samples over the course of multiple years while they also took assessments of PD severity using the UPDRS scale.

The reason why we are training only on the first month (visit_month == 0) is that the initial visit (month 0) provides a baseline measurement of PD severity for each patient. This baseline measurement can be used to train a model to predict the progression of PD over time using the protein abundance data.

By training only on the first month, we can ensure that the model is predicting the progression of PD rather than simply fitting the noise in the data. Additionally, by using the initial visit as the baseline, we can avoid the potential confounding effects of medication or other factors that may impact the UPDRS scores at later visits.Therefore, we extract the UPDRS scores for the initial visit (month 0) for each patient and use this as the target variable for training our predictive model.

In [11]:
df_0 = clinical[(clinical.visit_month == 0)][['visit_id','updrs_1']]
df_0

,visit_id,updrs_1
0,55_0,10.0
13,942_0,3.0
28,1517_0,11.0
38,1923_0,2.0
45,2660_0,2.0
...,...,...
2550,63875_0,3.0
2559,63889_0,7.0
2569,64669_0,12.0
2584,64674_0,5.0


In [12]:
peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
peptides

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.30
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.00
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.00
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.70
...,...,...,...,...,...,...
981829,58648_108,108,58648,Q9UHG2,ILAGSADSEGVAAPR,202820.00
981830,58648_108,108,58648,Q9UKV8,SGNIPAGTTVDTK,105830.00
981831,58648_108,108,58648,Q9Y646,LALLVDTVGPR,21257.60
981832,58648_108,108,58648,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26


In [13]:
peptides_test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv')
peptides_test

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,group_key
0,50423_0,0,50423,O00391,AHFSPSNIILDFPAAGSAAR,22226.30,0
1,50423_0,0,50423,O00391,NEQEQPLGQWHLS,10901.60,0
2,50423_0,0,50423,O00533,GNPEPTFSWTK,51499.40,0
3,50423_0,0,50423,O00533,IEIPSSVQQVPTIIK,125492.00,0
4,50423_0,0,50423,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,23174.20,0
...,...,...,...,...,...,...,...
2052,3342_6,6,3342,Q9Y646,AIINLAVYGK,6142.76,6
2053,3342_6,6,3342,Q9Y646,LALLVDTVGPR,23602.30,6
2054,3342_6,6,3342,Q9Y646,VGALASLIR,21728.20,6
2055,3342_6,6,3342,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,4253.17,6


### Peptides Features Engineering

In [14]:
peptides.groupby('visit_id').agg({'UniProt':'nunique','patient_id':'count','Peptide':'nunique','PeptideAbundance': ['min','max','mean','std']}).reset_index()

visit_id UniProt patient_id Peptide PeptideAbundance               \
               nunique      count nunique              min          max   
0      10053_0     165        649     649          82.9679   66333900.0   
1     10053_12     171        633     633         128.4460   73059300.0   
2     10053_18     208        868     868         108.5000   64711200.0   
3     10138_12     217        932     932         129.0240   71652400.0   
4     10138_24     219        918     918         142.6480  123897000.0   
...        ...     ...        ...     ...              ...          ...   
1108   8699_24     216        911     911         106.9420   99846400.0   
1109    942_12     212        889     889          88.3277   70888500.0   
1110    942_24     217        910     910         108.7050   71995500.0   
1111    942_48     216        907     907         148.1360   70658500.0   
1112     942_6     216        918     918         158.8690   58472500.0   

                                   
               mean           std  
0     726248.393431  3.535602e+06  
1     737183.385744  3.799654e+06  
2     601466.784320  3.006568e+06  
3     699099.199189  3.379573e+06  
4     732120.888877  4.912602e+06  
...             ...           ...  
1108  726494.824901  4.080307e+06  
1109  623193.979635  3.362987e+06  
1110  623849.652027  3.294163e+06  
1111  659297.802601  3.359265e+06  
1112  582795.558517  2.734257e+06  

[1113 rows x 8 columns]

The resulting DataFrame has the following columns:

visit_id: The ID code for the visit.
UniProt_nunique: The number of unique UniProt IDs in each group.
patient_id_count: The total number of patient IDs in each group.
Peptide_nunique: The number of unique peptide sequences in each group.
PeptideAbundance_min: The minimum PeptideAbundance in each group.
PeptideAbundance_max: The maximum PeptideAbundance in each group.
PeptideAbundance_mean: The mean PeptideAbundance in each group.
PeptideAbundance_std: The standard deviation of PeptideAbundance in each group.

By aggregating the data in this way, we can create features that capture different aspects of the protein abundance data for each patient visit, which can then be used as input features for a predictive model. This can help to reduce the dimensionality of the data, simplify the model, and potentially improve its performance.

In [15]:
peptides_PeptideAbundance_ft = peptides.groupby('visit_id').agg(Abe_min=('PeptideAbundance','min'), Abe_max=('PeptideAbundance','max'),\
                                                                Abe_mean=('PeptideAbundance','mean'), Abe_std=('PeptideAbundance','std'))\
                .reset_index()
peptides_PeptideAbundance_ft

,visit_id,Abe_min,Abe_max,Abe_mean,Abe_std
0,10053_0,82.9679,66333900.0,726248.393431,3.535602e+06
1,10053_12,128.4460,73059300.0,737183.385744,3.799654e+06
2,10053_18,108.5000,64711200.0,601466.784320,3.006568e+06
3,10138_12,129.0240,71652400.0,699099.199189,3.379573e+06
4,10138_24,142.6480,123897000.0,732120.888877,4.912602e+06
...,...,...,...,...,...
1108,8699_24,106.9420,99846400.0,726494.824901,4.080307e+06
1109,942_12,88.3277,70888500.0,623193.979635,3.362987e+06
1110,942_24,108.7050,71995500.0,623849.652027,3.294163e+06
1111,942_48,148.1360,70658500.0,659297.802601,3.359265e+06


In [16]:
df_peptides = pd.merge(peptides, df_0, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
peptides_PeptideAbundance_updrs

,Peptide,updrs_1_sum
0,AADDTWEPFASGK,5.357143
1,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,5.296703
2,AAFTEC(UniMod_4)C(UniMod_4)QAADK,5.305699
3,AANEVSSADVK,5.364780
4,AATGEC(UniMod_4)TATVGKR,5.146497
...,...,...
963,YVNKEIQNAVNGVK,5.319588
964,YWGVASFLQK,5.273684
965,YYC(UniMod_4)FQGNQFLR,5.336788
966,YYTYLIMNK,5.229508


This code is performing protein features engineering by extracting the peptide abundance values for each patient visit and calculating the mean UPDRS score for the initial visit (month 0) for each patient. The resulting dataframe contains two columns: Peptide and updrs_1_sum, which represents the mean UPDRS score for the initial visit for each peptide.

Here's a step-by-step explanation of the code:

The pd.merge() function is used to merge the peptides dataframe with the df_0 dataframe, which contains the UPDRS scores for the initial visit (month 0) for each patient. The merge is performed on the visit_id column and the merge type is set to inner. The resulting dataframe df_peptides contains only the peptides that have a corresponding UPDRS score for the initial visit.

The groupby() method is used to group the df_peptides dataframe by Peptide.

The agg() method is then used to apply an aggregation function to the updrs_1 column for each group. In this case, the aggregation function is mean(), which calculates the mean UPDRS score for the initial visit for each peptide.

The resulting dataframe peptides_PeptideAbundance_updrs contains two columns: Peptide and updrs_1_sum, which represents the mean UPDRS score for the initial visit for each peptide.

By calculating the mean UPDRS score for the initial visit for each peptide, we can potentially identify peptides that are associated with the early stages of Parkinson's disease. These peptides can then be used as input features for a predictive model to improve the accuracy of the predictions.

In [17]:
df_peptides = pd.merge(peptides, peptides_PeptideAbundance_updrs, on = 'Peptide', how = 'left')
peptides_ft = df_peptides.groupby('visit_id').agg(peptides_updrs_1_min=('updrs_1_sum','min'), peptides_updrs_1_max=('updrs_1_sum','max'),\
                                                          peptides_updrs_1_mean=('updrs_1_sum','mean'), peptides_updrs_1_std=('updrs_1_sum','std'))\
                .reset_index()
peptides_ft

,visit_id,peptides_updrs_1_min,peptides_updrs_1_max,peptides_updrs_1_mean,peptides_updrs_1_std
0,10053_0,4.878788,5.661972,5.279278,0.092880
1,10053_12,4.816794,5.661972,5.277513,0.097712
2,10053_18,4.297619,5.661972,5.265384,0.116303
3,10138_12,4.297619,5.661972,5.253513,0.126117
4,10138_24,4.297619,5.661972,5.257710,0.123452
...,...,...,...,...,...
1108,8699_24,4.572519,5.661972,5.256902,0.123395
1109,942_12,4.572519,5.661972,5.254323,0.118205
1110,942_24,4.572519,5.652174,5.255565,0.117226
1111,942_48,4.572519,5.652174,5.253489,0.119653


This code is performing additional protein feature engineering by calculating aggregate features based on the updrs_1_sum column for each visit in the df_peptides dataframe.

Here's a step-by-step explanation of the code:

The pd.merge() function is used to merge the peptides dataframe with the peptides_PeptideAbundance_updrs dataframe, which contains the mean UPDRS score for the initial visit (month 0) for each peptide. The merge is performed on the Peptide column and the merge type is set to left. The resulting dataframe df_peptides contains the peptide abundance values for each visit and the mean UPDRS score for the initial visit for each peptide.

The groupby() method is used to group the df_peptides dataframe by visit_id.

The agg() method is then used to apply aggregation functions to the updrs_1_sum column for each group. Four aggregation functions are used: min(), max(), mean(), and std(). The resulting dataframe peptides_ft contains four columns: visit_id, peptides_updrs_1_min, peptides_updrs_1_max, peptides_updrs_1_mean, and peptides_updrs_1_std. These columns represent the minimum, maximum, mean, and standard deviation of the mean UPDRS score for the initial visit for all peptides associated with each visit.

By calculating aggregate features based on the mean UPDRS score for the initial visit for each peptide, we can potentially identify trends and patterns in the relationship between protein abundance and Parkinson's disease progression. These features can then be used as input features for a predictive model to improve the accuracy of the predictions.

In [18]:
proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
proteins

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


### Proteins Features Engineering

In [19]:
proteins_test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv')
proteins_test

,visit_id,visit_month,patient_id,UniProt,NPX,group_key
0,50423_0,0,50423,O00391,33127.90,0
1,50423_0,0,50423,O00533,490742.00,0
2,50423_0,0,50423,O00584,43615.30,0
3,50423_0,0,50423,O14773,16486.60,0
4,50423_0,0,50423,O14791,2882.42,0
...,...,...,...,...,...,...
448,3342_6,6,3342,Q9UHG2,325226.00,6
449,3342_6,6,3342,Q9UKV8,64411.50,6
450,3342_6,6,3342,Q9UNU6,25117.50,6
451,3342_6,6,3342,Q9Y646,51473.30,6


In [20]:
proteins.groupby('visit_id').agg({'UniProt':'nunique','patient_id':'count','NPX':['min','max','mean','std']}).reset_index()

visit_id UniProt patient_id       NPX                             \
               nunique      count       min          max          mean   
0      10053_0     165        165  2497.840  269126000.0  2.856580e+06   
1     10053_12     171        171  5800.870  270030000.0  2.728871e+06   
2     10053_18     208        208  1334.110  278835000.0  2.509967e+06   
3     10138_12     217        217  2520.240  365582000.0  3.002583e+06   
4     10138_24     219        219  1436.940  396894000.0  3.068891e+06   
...        ...     ...        ...       ...          ...           ...   
1108   8699_24     216        216   756.551  346067000.0  3.064059e+06   
1109    942_12     212        212  1722.770  330558000.0  2.613298e+06   
1110    942_24     217        217  1339.150  336769000.0  2.616142e+06   
1111    942_48     216        216  1272.480  358059000.0  2.768442e+06   
1112     942_6     216        216  2491.690  290111000.0  2.476880e+06   

                    
               std  
0     2.131630e+07  
1     2.092162e+07  
2     1.969453e+07  
3     2.516170e+07  
4     2.716806e+07  
...            ...  
1108  2.409420e+07  
1109  2.295228e+07  
1110  2.312662e+07  
1111  2.460543e+07  
1112  2.002174e+07  

[1113 rows x 7 columns]

This code is performing protein feature engineering by calculating aggregate features based on the NPX column for each visit in the proteins dataframe.

Here's a step-by-step explanation of the code:

The groupby() method is used to group the proteins dataframe by visit_id.

The agg() method is then used to apply aggregation functions to the UniProt, patient_id, and NPX columns for each group. For the UniProt and patient_id columns, the aggregation function used is nunique(), which counts the number of unique values in each column. For the NPX column, four aggregation functions are used: min(), max(), mean(), and std(). The resulting dataframe contains four columns: visit_id, UniProt, patient_id, and NPX. These columns represent the number of unique proteins and patients associated with each visit, and the minimum, maximum, mean, and standard deviation of the normalized protein expression for all proteins associated with each visit.

By calculating aggregate features based on the normalized protein expression for each protein, we can potentially identify trends and patterns in the relationship between protein expression and Parkinson's disease progression. These features can then be used as input features for a predictive model to improve the accuracy of the predictions.

In [21]:
proteins_npx_ft = proteins.groupby('visit_id').agg(NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std'))\
                .reset_index()
proteins_npx_ft

,visit_id,NPX_min,NPX_max,NPX_mean,NPX_std
0,10053_0,2497.840,269126000.0,2.856580e+06,2.131630e+07
1,10053_12,5800.870,270030000.0,2.728871e+06,2.092162e+07
2,10053_18,1334.110,278835000.0,2.509967e+06,1.969453e+07
3,10138_12,2520.240,365582000.0,3.002583e+06,2.516170e+07
4,10138_24,1436.940,396894000.0,3.068891e+06,2.716806e+07
...,...,...,...,...,...
1108,8699_24,756.551,346067000.0,3.064059e+06,2.409420e+07
1109,942_12,1722.770,330558000.0,2.613298e+06,2.295228e+07
1110,942_24,1339.150,336769000.0,2.616142e+06,2.312662e+07
1111,942_48,1272.480,358059000.0,2.768442e+06,2.460543e+07


The code starts by grouping the proteins DataFrame by visit_id, and then aggregating the data for each group using the agg method. Specifically, the code calculates the minimum, maximum, mean, and standard deviation of the NPX column for each group, using the syntax NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std').

This results in a new DataFrame called proteins_npx_ft, which has one row per visit_id and columns for NPX_min, NPX_max, NPX_mean, and NPX_std. These columns represent features calculated from the NPX values for each visit.

The NPX value is the normalized protein expression frequency, which represents the frequency of the protein's occurrence in the sample. It may not have a one-to-one relationship with the component peptides as some proteins contain repeated copies of a given peptide. By aggregating these values at the visit_id level, we obtain features that describe the overall protein expression for each visit. These features may be useful in predicting the course of Parkinson's disease, as changes in protein expression are known to be associated with the disease.

Overall, this code is an example of feature engineering, which is the process of creating new features from existing data that may be useful in machine learning models. By generating these new features, we can potentially improve the accuracy of our predictions and gain insights into the underlying factors that contribute to Parkinson's disease.

In [22]:
df_proteins = pd.merge(proteins, df_0, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
proteins_Uniprot_updrs

,UniProt,updrs_1_sum
0,O00391,4.971014
1,O00533,5.319588
2,O00584,5.286458
3,O14498,5.217877
4,O14773,5.371585
...,...,...
222,Q9UHG2,5.319588
223,Q9UKV8,5.455090
224,Q9UNU6,5.296296
225,Q9Y646,5.266304


This code performs protein feature engineering. It starts by merging the proteins dataframe with the df_0 dataframe, which only contains the data from the first visit of each patient. The resulting merged dataframe df_proteins will only include proteins that were observed in the first visit of a patient.

Next, the code groups the merged dataframe df_proteins by UniProt, which is the UniProt ID code for the associated protein, and calculates the mean of updrs_1 for each unique UniProt value. This is done using the agg() function and the mean() method on the updrs_1 column. The resulting dataframe proteins_Uniprot_updrs shows the average updrs_1 score for each unique protein in the dataset.

This type of feature engineering could help identify proteins that are associated with PD and are potential biomarkers for the disease. By analyzing the relationship between the protein expression levels and the UPDRS score, which is a measure of motor function in PD, we can gain insights into the underlying mechanisms of the disease and potentially identify targets for therapy.

In [23]:
df_proteins = pd.merge(proteins, proteins_Uniprot_updrs, on = 'UniProt', how = 'left')
proteins_UniProt_ft = df_proteins.groupby('visit_id').agg(proteins_updrs_1_min=('updrs_1_sum','min'), proteins_updrs_1_max=('updrs_1_sum','max'),\
                                                          proteins_updrs_1_mean=('updrs_1_sum','mean'), proteins_updrs_1_std=('updrs_1_sum','std'))\
                .reset_index()
proteins_UniProt_ft

,visit_id,proteins_updrs_1_min,proteins_updrs_1_max,proteins_updrs_1_mean,proteins_updrs_1_std
0,10053_0,4.892857,5.601449,5.300548,0.077355
1,10053_12,4.816794,5.652174,5.296073,0.099055
2,10053_18,4.297619,5.652174,5.272617,0.134631
3,10138_12,4.297619,5.652174,5.263118,0.143238
4,10138_24,4.297619,5.652174,5.269522,0.137776
...,...,...,...,...,...
1108,8699_24,4.643939,5.652174,5.272980,0.126708
1109,942_12,4.816794,5.652174,5.273887,0.109774
1110,942_24,4.643939,5.652174,5.268376,0.125426
1111,942_48,4.643939,5.652174,5.269472,0.125393


In the first line, the proteins dataframe is merged with the df_0 dataframe using the visit_id column as the common key. The resulting merged dataframe is stored in df_proteins. This operation ensures that we only consider the protein expression data from the first visit of each patient, which is stored in df_0.

Next, the df_proteins dataframe is grouped by the UniProt column, and the mean value of updrs_1 column is calculated for each unique value of UniProt. The resulting dataframe is stored in proteins_Uniprot_updrs. This step aggregates the UPDRS score information across all patients and visits associated with each unique protein.

In the third line, the proteins dataframe is merged with proteins_Uniprot_updrs using the UniProt column as the common key. The resulting merged dataframe is stored in df_proteins. This operation ensures that we have the UPDRS score information for each unique protein.

Next, the df_proteins dataframe is grouped by the visit_id column, and the minimum, maximum, mean, and standard deviation of updrs_1_sum column are calculated. The resulting dataframe is stored in proteins_UniProt_ft. This step aggregates the UPDRS score information across all unique proteins associated with each visit, which can be used as features for downstream modeling tasks.

Overall, these steps are used to engineer protein-level features that capture the relationship between protein expression levels and UPDRS scores, which can be useful for predicting PD progression or assessing treatment efficacy.

In [24]:
test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv')
test

,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0
5,50423_0,0,50423,updrs_2,50423_0_updrs_2,0
6,50423_0,0,50423,updrs_3,50423_0_updrs_3,0
7,50423_0,0,50423,updrs_4,50423_0_updrs_4,0
8,3342_6,6,3342,updrs_1,3342_6_updrs_1,6
9,3342_6,6,3342,updrs_2,3342_6_updrs_2,6


## Merge Data

In [25]:
df_0_1 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_1']]
df_0_2 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_2']]
df_0_3 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_3']]
df_0_4 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_4']]

df_proteins = pd.merge(proteins, df_0_1, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs1 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_2, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs2 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_2','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_3, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs3 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_3','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_4, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs4 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_4','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_1, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs1 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_2, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs2 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_2','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_3, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs3 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_3','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_4, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs4 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_4','mean')).reset_index()

df_proteins_fts = [proteins_Uniprot_updrs1, proteins_Uniprot_updrs2, proteins_Uniprot_updrs3, proteins_Uniprot_updrs4]
df_peptides_fts = [peptides_PeptideAbundance_updrs1, peptides_PeptideAbundance_updrs2, peptides_PeptideAbundance_updrs3, peptides_PeptideAbundance_updrs4]
df_lst = [df_0_1, df_0_2, df_0_3, df_0_4]

The given code merges the data in this way to calculate the average UPDRS scores associated with each protein and peptide. By merging the data, it allows the relationship between the protein/peptide levels and the UPDRS scores to be analyzed. This analysis can be beneficial in understanding the progression of Parkinson's disease and predicting its course using protein abundance data.

Here's a breakdown of the code and its purpose:

Four separate data frames (df_0_1, df_0_2, df_0_3, df_0_4) are created, each containing the visit_id and a specific UPDRS score (updrs_1, updrs_2, updrs_3, or updrs_4) for patients at visit_month == 3.

For each UPDRS score, the 'proteins' data frame is merged with the corresponding UPDRS score data frame (e.g., df_0_1) based on the 'visit_id' column. This allows the code to link the protein data with the UPDRS scores for each patient visit.

After merging the data, the 'proteins_Uniprot_updrsN' data frames are created by grouping the merged data by the 'UniProt' column and calculating the mean of the corresponding UPDRS score for each group. This results in a data frame containing the average UPDRS score for each protein.

A similar process is performed for peptides. The 'peptides' data frame is merged with the UPDRS score data frames, and the 'peptides_PeptideAbundance_updrsN' data frames are created by grouping the merged data by the 'Peptide' column and calculating the mean of the corresponding UPDRS score for each group. This results in a data frame containing the average UPDRS score for each peptide.

In [26]:
def features(df, proteins, peptides, classes):
    proteins_npx_ft = proteins.groupby('visit_id').agg(NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std'))\
                    .reset_index()
    peptides_PeptideAbundance_ft = peptides.groupby('visit_id').agg(Abe_min=('PeptideAbundance','min'), Abe_max=('PeptideAbundance','max'),\
                                                                    Abe_mean=('PeptideAbundance','mean'), Abe_std=('PeptideAbundance','std'))\
                    .reset_index()

    df_proteins = pd.merge(proteins, df_proteins_fts[classes], on = 'UniProt', how = 'left')
    proteins_UniProt_ft = df_proteins.groupby('visit_id').agg(proteins_updrs_1_min=('updrs_1_sum','min'), proteins_updrs_1_max=('updrs_1_sum','max'),\
                                                              proteins_updrs_1_mean=('updrs_1_sum','mean'), proteins_updrs_1_std=('updrs_1_sum','std'))\
                    .reset_index()
    df_peptides = pd.merge(peptides, df_peptides_fts[classes], on = 'Peptide', how = 'left')
    peptides_ft = df_peptides.groupby('visit_id').agg(peptides_updrs_1_min=('updrs_1_sum','min'), peptides_updrs_1_max=('updrs_1_sum','max'),\
                                                              peptides_updrs_1_mean=('updrs_1_sum','mean'), peptides_updrs_1_std=('updrs_1_sum','std'))\
                    .reset_index()

    df = pd.merge(df, proteins_npx_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, peptides_PeptideAbundance_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, proteins_UniProt_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, peptides_ft, on = 'visit_id', how = 'left')
    df = df.fillna(df.mean())
    return df

The purpose of this function features is to create an enriched dataset by merging various aggregated features from the protein and peptide data with the main input dataframe (df). This merged dataframe will then be used for further analysis, such as training a machine learning model. Here's the step-by-step explanation of the code and the reasoning behind merging the data in this way:

Calculate aggregated features for proteins and peptides:

proteins_npx_ft: Group the proteins dataframe by 'visit_id' and calculate the minimum, maximum, mean, and standard deviation of the 'NPX' column for each group.

peptides_PeptideAbundance_ft: Group the peptides dataframe by 'visit_id' and calculate the minimum, maximum, mean, and standard deviation of the 'PeptideAbundance' column for each group.

Merge aggregated features (from df_proteins_fts and df_peptides_fts) with the proteins and peptides data:

df_proteins: Merge the proteins dataframe with the aggregated features dataframe df_proteins_fts[classes] based on the 'UniProt' column.

df_peptides: Merge the peptides dataframe withthe aggregated features dataframe df_peptides_fts[classes] based on the 'Peptide' column.

Calculate aggregated features for the merged protein and peptide data:

proteins_UniProt_ft: Group the df_proteins dataframe by 'visit_id' and calculate the minimum, maximum, mean, and standard deviation of the UPDRS score column for each group.

peptides_ft: Group the df_peptides dataframe by 'visit_id' and calculate the minimum, maximum, mean, and standard deviation of the UPDRS score column for each group.

Merge all the aggregated features with the main input dataframe (df):

Merge df with proteins_npx_ft, peptides_PeptideAbundance_ft, proteins_UniProt_ft, and peptides_ft dataframes based on the 'visit_id' column.

Fill missing values in the merged dataframe (df) with the mean of the respective columns.

By merging the data in this way, the function creates a more comprehensive set of features that capture the relationship between the protein/peptide levels and the UPDRS scores. These features can then be used in a machine learning model to predict the progression of Parkinson's disease based on the protein abundance data.



In [27]:
train_0 = features(df_0_1, proteins, peptides, 0)
train_0

,visit_id,updrs_1,NPX_min,NPX_max,NPX_mean,NPX_std,Abe_min,Abe_max,Abe_mean,Abe_std,proteins_updrs_1_min,proteins_updrs_1_max,proteins_updrs_1_mean,proteins_updrs_1_std,peptides_updrs_1_min,peptides_updrs_1_max,peptides_updrs_1_mean,peptides_updrs_1_std
0,55_3,10.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
1,942_3,7.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
2,3636_3,4.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
3,4161_3,1.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
4,5645_3,5.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,62723_3,7.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
111,62792_3,0.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
112,64669_3,15.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
113,64674_3,5.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0


## Metric SMAPE

In [28]:
# Get the min and max for each updrs variable
updrs_vars = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']
updrs_min_max = pd.DataFrame({'Min': clinical[updrs_vars].min(), 'Max': clinical[updrs_vars].max()})
print(updrs_min_max)

         Min   Max
updrs_1  0.0  33.0
updrs_2  0.0  40.0
updrs_3  0.0  86.0
updrs_4  0.0  20.0


SMAPE is a popular metric for time series forecasting problems, and it is often used to evaluate the performance of machine learning models in such scenarios. Unlike MSE and RMSE, SMAPE is a relative error metric that takes into account the magnitude of the actual values. This makes SMAPE more suitable for evaluating the accuracy of predictions when the actual values can vary significantly in magnitude.

In the case of this competition, the UPDRS scores can range from 0 to 86, so the actual values can vary greatly in magnitude. SMAPE calculates the percentage error between the predicted and actual values, which normalizes the error by the magnitude of the actual values. This makes it easier to compare the accuracy of the predictions across different UPDRS scores.

MSE and RMSE, on the other hand, are absolute error metrics that do not take into account the magnitude of the actual values. As a result, they may not be as effective in evaluating the accuracy of predictions when the actual values can vary significantly in magnitude.

In [29]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

In this case, MinMaxScaler, a preprocessing technique, is used for several reasons:

Algorithm Requirements: The model selection process includes various algorithms like K-Nearest Neighbors and Gradient Boosting. Both of these algorithms are sensitive to the scale of input features, and their performance can be affected if the features are on different scales. MinMax scaling ensures that all features have the same scale, which can improve the performance of these algorithms and help them find patterns in the data more efficiently.

Non-normal distribution: It's mentioned that the train_0 data is not normally distributed. MinMaxScaler does not assume a normal distribution of the data, making it more appropriate for this case than StandardScaler, which assumes a normal distribution.

Consistency across different models: Since multiple models are being used in this code (Lasso Regression, Random Forest, Gradient Boosting, K-Nearest Neighbors, Decision Tree, and Stacking model), using MinMaxScaler provides a consistent scaling approach across all of these models, making the results more comparable and easier to interpret.

In [30]:
def objective(trial, X, y, model_name):
    if model_name == 'lr':
        model = Lasso(
        alpha=trial.suggest_float('alpha', 0.01, 0.1, step=0.01)
      )

    elif model_name == 'rf':
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int('n_estimators', 5, 100),
            max_features=trial.suggest_categorical('max_features', ['auto', 'sqrt']),
            max_depth=trial.suggest_int('max_depth', 10, 120),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 11),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
            bootstrap=trial.suggest_categorical('bootstrap', [True, False])
        )
        
    elif model_name == 'gb':
        model=GradientBoostingRegressor(
            n_estimators= trial.suggest_int("n_estimators", 100, 5000, step = 100),
            learning_rate= trial.suggest_float("learning_rate", 1e-4, 0.3, log = True),
            max_depth= trial.suggest_int("max_depth", 3, 9),
            subsample=trial.suggest_float("subsample", 0.5, 0.9, step = 0.1),
            max_features= trial.suggest_categorical("max_features", ["auto", "sqrt", "log2"]),
        )
        
    elif model_name == 'knn':
        model = KNeighborsRegressor(
             n_neighbors=trial.suggest_int('n_neighbors', 1, 50),
             weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
             algorithm=trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']),
             leaf_size=trial.suggest_int('leaf_size', 10, 50),
             p=trial.suggest_int('p', 1, 5),
    )

    elif model_name == 'dt':
        model = DecisionTreeRegressor(
            max_features=trial.suggest_categorical('max_features', ['auto', 'sqrt']),
            max_depth=trial.suggest_int('max_depth', 10, 120),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 11),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10)
        )

    score = -cross_val_score(model, X, y, cv=cv, scoring=make_scorer(smape), n_jobs=-1).mean()
    return score

# Preprocessing Data
model = {}
mms = MinMaxScaler()

for i in range(3):
    print(f"\n\n{m_}########## Iteration: {c_}{i + 1}{m_} ##########")
    cv = KFold(n_splits = 10, shuffle=True, random_state=42)
    
    train_0 = features(df_lst[i], proteins, peptides, i)
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std']
    train_0[scale_col] = mms.fit_transform(train_0[scale_col])
    
    X = train_0.drop(columns = ['visit_id', 'updrs_{0}'.format(i + 1)], axis=1)
    y = train_0['updrs_{0}'.format(i + 1)].astype(np.float32)
    
    sampler = RandomSampler(seed=42)

    # Lasso Regression
    print(g_,"Lasso Regression HPO")
    lr_study = optuna.create_study(direction="minimize", sampler=sampler)
    lr_study.optimize(lambda trial: objective(trial, X, y, 'lr'), n_trials=100)
    lr_best_params = lr_study.best_params
    
    # Random Forest
    print(g_,"Random Forest HPO")
    rf_study = optuna.create_study(direction="minimize", sampler=sampler)
    rf_study.optimize(lambda trial: objective(trial, X, y, 'rf'), n_trials=100)
    rf_best_params = rf_study.best_params
    
    # Gradient Boosting 
    print(g_,"Gradient Boosting HPO")
    gb_study = optuna.create_study(direction="minimize", sampler=sampler)
    gb_study.optimize(lambda trial: objective(trial, X, y, 'gb'), n_trials=100)
    gb_best_params = gb_study.best_params
   
    # K-Nearest Neighbors
    print(g_,"K-Nearest Neighbors HPO")
    knn_study = optuna.create_study(direction="minimize", sampler=sampler)
    knn_study.optimize(lambda trial: objective(trial, X, y, 'knn'), n_trials=100)
    knn_best_params = knn_study.best_params
    
    # Decision Tree
    print(g_,"Decision Tree HPO")
    dt_study = optuna.create_study(direction="minimize", sampler=sampler)
    dt_study.optimize(lambda trial: objective(trial, X, y, 'dt'), n_trials=100)
    dt_best_params = dt_study.best_params
    
    # Instantiate the base models with the best hyperparameters found by Optuna
    lr_best = Lasso(**lr_best_params)
    rf_best = RandomForestRegressor(**rf_best_params)
    gb_best= GradientBoostingRegressor(**gb_best_params)
    knn_best = KNeighborsRegressor(**knn_best_params)
    dt_best = DecisionTreeRegressor(**dt_best_params)
    
    lr_best.fit(X, y)
    rf_best.fit(X, y)
    gb_best.fit(X,y)
    knn_best.fit(X, y)
    dt_best.fit(X, y)
    
    # Define the base models as a list of tuples (name, model)
    base_models = [
        ('lr', lr_best),
        ('rf', rf_best),
        ('gb', gb_best),
        ('knn',knn_best),
        ('dt', dt_best)
    ]
    
    # Instantiate the meta-model using LightGBM
    meta_model = lgb.LGBMRegressor()
    
    # Create the stacking model
    stacking_model = StackingRegressor(
        estimators = base_models, 
        final_estimator = meta_model, 
        cv = cv, 
        n_jobs = -1
    )
    
    # Fit the stacking model
    stacking_model.fit(X, y)
    print(f'{b_}Train SMAPE Lasso Regression:',r_,smape(train_0['updrs_{0}'.format(i + 1)], lr_best.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print(f'{b_}Train SMAPE Random Forest:',r_,smape(train_0['updrs_{0}'.format(i + 1)], rf_best.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print(f'{b_}Train SMAPE Gradient Boosting :',r_,smape(train_0['updrs_{0}'.format(i + 1)], gb_best.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print(f'{b_}Train SMAPE K-Nearest Neighbors:',r_,smape(train_0['updrs_{0}'.format(i + 1)], knn_best.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    
    print(f'{b_}Train SMAPE decision tree:',r_,smape(train_0['updrs_{0}'.format(i + 1)], dt_best.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print(f'{b_}Train smape stacking model:',r_,smape(train_0['updrs_{0}'.format(i + 1)], stacking_model.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
                                                           
    model['lr_' + str(i)] = lr_best
    model['rf_' + str(i)] = rf_best
    model['gb_' + str(i)] = gb_best
    model['knn_' + str(i)] = knn_best
    model['dt_' + str(i)] = dt_best
    model['stack_' + str(i)] = stacking_model



########## Iteration: 1 ##########
 Lasso Regression HPO
 Random Forest HPO
 Gradient Boosting HPO
 K-Nearest Neighbors HPO
 Decision Tree HPO
Train SMAPE Lasso Regression:  70.46046849710056
Train SMAPE Random Forest:  70.3015330886209
Train SMAPE Gradient Boosting :  70.39295333588805
Train SMAPE K-Nearest Neighbors:  87.56335452694535
Train SMAPE decision tree:  70.46046849710056
Train smape stacking model:  70.71551532380823


########## Iteration: 2 ##########
 Lasso Regression HPO
 Random Forest HPO
 Gradient Boosting HPO
 K-Nearest Neighbors HPO
 Decision Tree HPO
Train SMAPE Lasso Regression:  64.58294234685054
Train SMAPE Random Forest:  64.47778270703644
Train SMAPE Gradient Boosting :  64.4897285460952
Train SMAPE K-Nearest Neighbors:  96.7930133594984
Train SMAPE decision tree:  64.58294234685054
Train smape stacking model:  64.63896131117143


########## Iteration: 3 ##########
 Lasso Regression HPO
 Random Forest HPO
 Gradient Boosting HPO
 K-Nearest Neighbors HPO
 Deci

HPO stands for Hyperparameter Optimization. Hyperparameters are settings that need to be configured by the user, such as the learning rate or the number of trees in a random forest model. Hyperparameter optimization is the process of finding the best values for these hyperparameters that maximize the performance of the model on the given data.

HPO is used to optimize the hyperparameters of the base models, including Lasso Regression, Random Forest, Gradient Boosting, K-Nearest Neighbors, and Decision Tree. Optuna is used to perform the HPO process in this code. By optimizing the hyperparameters of these models, the aim is to improve their performance and enable the stacked regression model to make more accurate predictions on the given dataset.

This code consists of a function objective and a loop that iterates three times. The main purpose of the code is to find the best hyperparameters for several machine learning models, train them on the data, and create an ensemble stacking model for each iteration. The models include Lasso Regression, Random Forest, Gradient Boosting, K-Nearest Neighbors, and Decision Tree.

objective: This function is used by Optuna to optimize the hyperparameters of the given models. It takes as input a trial object, the input features X, the target variable y, and the model name. Depending on the model name, it creates a corresponding model with hyperparameters sampled by Optuna. It then evaluates the model using cross-validation and returns the mean of the cross-validated SMAPE (Symmetric Mean Absolute Percentage Error) scores.

The outer loop iterates three times, each time performing the following steps:

a. Print the iteration number.

b. Create a 10-fold cross-validator object with a random seed.

c. Preprocess the data by calling the features function, which extracts and aggregates various features from the protein and peptide data. It then scales some of the columns using MinMaxScaler.

d. Separate the input features X and target variable y from the preprocessed data.

e. Create an Optuna sampler object with a random seed.

f. For each of the five base models (Lasso Regression, Random Forest, Gradient Boosting, K-Nearest Neighbors, and Decision Tree), use Optuna to find the best hyperparameters. This is done by creating an Optuna study and optimizing the objective function, passing the corresponding model name. Save the best hyperparameters found for each model.

g. Instantiate the base models using the best hyperparameters found by Optuna and fit them on the data.

h. Create a list of tuples containing the name and the instantiated base model.

i. Instantiate a meta-model using LightGBM.

j. Create a stacking model using the base models and the meta-model, specifying the cross-validator and number of jobs.

k. Fit the stacking model on the data.

l. Calculate and print the train SMAPE scores for each of the base models and the stacking model.

m. Store the trained models in a dictionary for later use.

The code implements a stacked regression model that combines the predictions of five different base models, including Lasso Regression, Random Forest, Gradient Boosting, K-Nearest Neighbors, and Decision Tree. The aim is to make accurate predictions on the given dataset. To achieve this, Optuna is used for hyperparameter optimization of the base models, and the stacked regression model's performance is evaluated on the training data using SMAPE.

Among the base models, Random Forest has the lowest SMAPE, indicating its superior performance in making predictions on the data. Lasso Regression and Gradient Boosting also have relatively low SMAPE values and perform similarly to each other. However, K-Nearest Neighbors has a much higher SMAPE, suggesting that it performs poorly in making predictions on the data.

The stacked regression model's performance is also evaluated using SMAPE, and it has a slightly higher SMAPE than the best performing base model (Random Forest). Nevertheless, it still performs well and is likely to make accurate predictions on new data. LightGBM is used as the final estimator in the stacking model because of its ability to handle categorical features and learn complex nonlinear relationships in the data efficiently and effectively.

Overall, the stacked regression model is expected to provide accurate predictions on new data, given the good performance of the individual base models on the training data.

## Plot Visuals From Optuna Hyperparameter Optimization

### plot_optimization_history

The plot_optimization_history function is a visualization tool provided by Optuna that displays the optimization history of a hyperparameter optimization study. The function creates a graph with the iteration number on the x-axis and the SMAPE value on the y-axis.

The graph is composed of blue dots that represent the SMAPE value for each trial conducted during the optimization process. The red line represents the best SMAPE value attained during the optimization process. By examining the graph, one can visually analyze the performance of the optimization process and determine whether there were any trends or patterns in the results.

This function is useful for understanding the optimization process's performance and evaluating the effectiveness of the chosen hyperparameters for a given model. The best SMAPE value attained is easily identifiable, allowing one to determine the optimization process's success in finding the optimal hyperparameters for the given model.

### plot_param_importances

The plot_param_importances function is a visualization tool provided by Optuna that displays the relative importances of the hyperparameters in a hyperparameter optimization study. The function creates a graph with the hyperparameters on the y-axis and their relative importances on the x-axis.

The graph is composed of bars that represent the relative importances of the hyperparameters. The taller the bar, the more important the hyperparameter is in terms of its contribution to the optimization process. By examining the graph, one can visually analyze the relative importance of each hyperparameter and determine which ones were the most influential in achieving the optimal hyperparameters for the given model.

This function is useful for understanding which hyperparameters were the most influential in the optimization process and identifying areas where further optimization may be necessary. By identifying the most important hyperparameters, data scientists and machine learning practitioners can focus their efforts on optimizing those hyperparameters and achieving even better results in future hyperparameter optimization studies.

Optimization History for Lasso Regression Study

In [34]:
plot_optimization_history(lr_study)

Optimization History for Random Forest Study

In [36]:
plot_optimization_history(rf_study)

Hyperparameters importance of Random Forest parameters

In [37]:
plot_param_importances(rf_study)

Optimization History for Gradient Boosting Study

In [38]:
plot_optimization_history(gb_study)

Hyperparameters importance of Gradient Boosting parameters

In [39]:
plot_param_importances(gb_study)

Optimization History for K-Nearest Neighbors Study

In [40]:
plot_optimization_history(knn_study)

Hyperparameters importance of K-Nearest Neighbors parameters

In [41]:
plot_param_importances(knn_study)

Optimization History for Decision Tree Study

In [42]:
plot_optimization_history(dt_study)

Hyperparameters importance of Decision Tree parameters

In [43]:
plot_param_importances(dt_study)

In [32]:
import amp_pd_peptide
env = amp_pd_peptide.make_env()
iter_test = env.iter_test()

In [33]:
def map_test(x):
    updrs = x.split('_')[2] + '_' + x.split('_')[3]
    month = int(x.split('_plus_')[1].split('_')[0])
    visit_id = x.split('_')[0] + '_' + x.split('_')[1]
    # set all predictions 0 where updrs equals 'updrs_4'
    if updrs=='updrs_3':
#         rating = updrs_3_pred[month]
        rating = df[df.visit_id == visit_id]['pred2'].values[0]
    elif updrs=='updrs_4':
        rating = 0
    elif updrs =='updrs_1':
        rating = df[df.visit_id == visit_id]['pred0'].values[0]
    else:
        rating = df[df.visit_id == visit_id]['pred1'].values[0]
    return rating

counter = 0
# The API will deliver four dataframes in this specific order:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    df = test[['visit_id']].drop_duplicates('visit_id')
    pred_0 = features(df[['visit_id']], test_proteins, test_peptides, 0)
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std']
    pred_0[scale_col] = mms.fit_transform(pred_0[scale_col])
    pred_0 = model["stack_0"].predict(pred_0.drop(columns = ['visit_id'], axis = 1))
    df['pred0'] = np.ceil(pred_0 + 0)
    
    pred_1 = features(df[['visit_id']], test_proteins, test_peptides, 1)
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std']
    pred_1[scale_col] = mms.fit_transform(pred_1[scale_col])
    pred_1 = model["stack_1"].predict(pred_1.drop(columns = ['visit_id'], axis = 1))
    df['pred1'] = np.ceil(pred_1 + 0.5)
    
    pred_2 = features(df[['visit_id']], test_proteins, test_peptides, 2)
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std']
    pred_2[scale_col] = mms.fit_transform(pred_2[scale_col])
    pred_2 = model["stack_2"].predict(pred_2.drop(columns = ['visit_id'], axis = 1))
    df['pred2'] = np.ceil(pred_2 + 1.5)
    
    sample_submission['rating'] = sample_submission['prediction_id'].apply(map_test)
    env.predict(sample_submission)
    
    if counter == 0:
        display(test)
        display(sample_submission)
        
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,visit_id,visit_month,patient_id,updrs_test,row_id
0,3342_0,0,3342,updrs_1,3342_0_updrs_1
1,3342_0,0,3342,updrs_2,3342_0_updrs_2
2,3342_0,0,3342,updrs_3,3342_0_updrs_3
3,3342_0,0,3342,updrs_4,3342_0_updrs_4
4,50423_0,0,50423,updrs_1,50423_0_updrs_1
5,50423_0,0,50423,updrs_2,50423_0_updrs_2
6,50423_0,0,50423,updrs_3,50423_0_updrs_3
7,50423_0,0,50423,updrs_4,50423_0_updrs_4


,prediction_id,rating
0,3342_0_updrs_1_plus_0_months,6.0
1,3342_0_updrs_1_plus_6_months,6.0
2,3342_0_updrs_1_plus_12_months,6.0
3,3342_0_updrs_1_plus_24_months,6.0
4,3342_0_updrs_2_plus_0_months,8.0
5,3342_0_updrs_2_plus_6_months,8.0
6,3342_0_updrs_2_plus_12_months,8.0
7,3342_0_updrs_2_plus_24_months,8.0
8,3342_0_updrs_3_plus_0_months,22.0
9,3342_0_updrs_3_plus_6_months,22.0
